In [1]:
import sys, os
import numpy as np
import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, LineString, Polygon

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
# flags people that use any link of 101 between the county line and 101/280 interchange, 
# or any link of 280 between 101/280 interchange and northern terminus

In [3]:
# Street map 
OSM = r'Q:\GIS\Transportation\Roads\Bay_Area\OSM\osm.shp'

PATHS = r'..\data\survey\wkday\path.csv'
TRIPS = r'..\wkday\trip.csv'
FAC = r'..\data\support\101_280_by_segment.csv'
OUTDIR = r'..\data\survey\wkday'
#CLASS_IDS_TO_REMOVE = [202,120,116,121,118,119,117,114,122,113]
CLASS_IDS_TO_REMOVE = []
MODE_NUM_TO_NAME = {1:'walk',2:'bike',3:'da',4:'sr2',5:'sr3',6:'trn',
                    8:'schbus',11:'tnc1',12:'tnc2',13:'tnc3'}

In [4]:
writer = pd.ExcelWriter(os.path.join(OUTDIR, 'osm_vol_with_ci.xlsx'))

In [5]:
def sum_w_squared(w):
    w = w/w.sum()
    return w.pow(2).sum()

In [6]:
osm = gpd.read_file(OSM)
if len(CLASS_IDS_TO_REMOVE) > 0:
    osm = osm.loc[~osm['CLASS_ID'].isin(CLASS_IDS_TO_REMOVE)]

In [7]:
df = pd.read_csv(PATHS)
df.sort_values(by=['hh_id','person_id','trip_id','collected_at'], inplace=True)
fac = pd.read_csv(FAC)
df = pd.merge(df, fac[['gid','segment','route','split','dir']].rename(columns={'gid':'edge'}),
              on='edge', how='left')

In [8]:
osm = pd.merge(osm, fac[['gid','segment','route','split','dir']].rename(columns={'gid':'GID'}), 
               on='GID', how='left')

In [9]:
trips = pd.read_csv(TRIPS)

In [10]:
trips = trips.loc[trips['mode'].ne(0)]
trips['mode2'] = trips['mode']
trips.loc[trips['mode'].eq(9),'mode2'] = trips['dorp']
trips.loc[trips['mode'].eq(9) & trips['dorp'].eq(9), 'mode2'] = 11
# just use one transit mode for aggregation
trips.loc[trips['mode'].isin([7,70,71]), 'mode2'] = 6

In [11]:
trips['trip_id'] = trips['trip_id'].round().astype('int64')

In [12]:
links = df[['edge','trip_id']].drop_duplicates().sort_values(['edge','trip_id'])
links = pd.merge(links, trips, on='trip_id', how='left')
samp = links.groupby(['tp','edge']).size()
samp2 = pd.DataFrame(links.groupby('edge').size())
samp2['tp'] = 'daily'
samp2 = samp2.reset_index().set_index(['tp','edge'])
samp = pd.concat([samp,samp2])[0] # make it a series again with "[0]"
l1 = links.pivot_table(index=['tp','edge'], columns='mode2', values='trexpfac', aggfunc='sum', fill_value=0)
l2 = links.pivot_table(index=['edge'], columns='mode2', values='trexpfac', aggfunc='sum', fill_value=0)
l2['tp'] = 'daily'
l2.set_index('tp', append=True, inplace=True)
l2.index = l2.index.swaplevel()
tot = pd.concat([l1, l2])
shr = tot.divide(tot.sum(axis=1), axis='rows')

In [13]:
l1_w_term = links.groupby(['tp','edge']).agg({'trexpfac':sum_w_squared})
l2_w_term = links.groupby(['edge']).agg({'trexpfac':sum_w_squared})
l2_w_term['tp'] = 'daily'
l2_w_term.set_index('tp', append=True, inplace=True)
l2_w_term.index = l2_w_term.index.swaplevel()
wtrm = pd.concat([l1_w_term, l2_w_term])

In [14]:
wtrm = pd.Series(index=wtrm.index, data=wtrm['trexpfac'])

In [15]:
ci = 1.96 * (shr * (1-shr)).multiply(wtrm, axis='rows').pow(0.5)

In [16]:
tot.rename(columns=MODE_NUM_TO_NAME, inplace=True)
shr.rename(columns=MODE_NUM_TO_NAME, inplace=True)
ci.rename(columns=MODE_NUM_TO_NAME, inplace=True)

In [17]:
shr_lo = (shr - ci)
shr_lo[shr_lo.le(0)] = 0
shr_hi = shr + ci
tot_lo = shr_lo.multiply(tot.sum(axis=1), axis='rows')
tot_hi = shr_hi.multiply(tot.sum(axis=1), axis='rows')

In [18]:
shr['bound'] = 'mid'
shr_lo['bound'] = 'lower'
shr_hi['bound'] = 'upper'

tot['bound'] = 'mid'
tot_lo['bound'] = 'lower'
tot_hi['bound'] = 'upper'

In [19]:
tot['veh'] = tot['da'] + tot['tnc1'] + (tot['sr2'] + tot['tnc2']) / 2 + (tot['sr3'] + tot['tnc3']) / 3.5
tot_lo['veh'] = tot['veh']
tot_hi['veh'] = tot['veh']
try:
    tot['pax'] = tot[['da','sr2','sr3','trn','schbus','tnc1','tnc2','tnc3']].sum(axis=1)
except:
    print('no schbus')
    tot['pax'] = tot[['da','sr2','sr3','trn','tnc1','tnc2','tnc3']].sum(axis=1)
    
tot_lo['pax'] = tot['pax']
tot_hi['pax'] = tot['pax']

In [20]:
ci['samples'] = samp
ci['w_term'] = wtrm

In [21]:
tot = pd.concat([tot_lo, tot, tot_hi])
tot.set_index('bound', append=True, inplace=True)

shr = pd.concat([shr_lo, shr, shr_hi])
shr.set_index('bound', append=True, inplace=True)

In [22]:
veh_shr = tot[['da','sr2','sr3','tnc1','tnc2','tnc3']]
veh_shr = veh_shr.divide(tot['pax'], axis='rows')

In [23]:
tot.to_csv(os.path.join(OUTDIR,'osm_vol_with_ci.csv'))

In [24]:
tot['samples'] = samp

In [25]:
for tp in ['EA','AM','MD','PM','EV1','EV2']:
    osm_tp = pd.merge(osm, tot.loc[tp], left_on='GID', right_on='edge')
    osm_tp.to_file(R'Q:\Model Projects\101_280\data\osm\loaded_net_{}_survey.shp'.format(tp.lower()))

In [26]:
segment_volumes = pd.merge(osm.loc[pd.notnull(osm['segment'])],
                            tot.reset_index(),
                            left_on='GID',
                            right_on='edge')
segment_volumes.sort_values(by=['segment','tp'], inplace=True)
segment_volumes.to_csv(r'Q:\Model Projects\101_280\data\survey\osm_survey_mode_volumes_with_ci.csv', index=False)

In [27]:
segment_volumes

,GID,CLASS_ID,LENGTH,LENGTH_M,NAME,SOURCE,TARGET,X1,Y1,X2,...,sr2,sr3,trn,schbus,tnc1,tnc2,tnc3,veh,pax,samples
42,362374.0,101,0.004746,495.489572,James Lick Freeway,298081.0,115146.0,-122.407687,37.740903,-122.405078,...,2581.407981,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,8447.750707,14232.631583,58
49,362374.0,101,0.004746,495.489572,James Lick Freeway,298081.0,115146.0,-122.407687,37.740903,-122.405078,...,6841.604825,3309.709850,0.000000,0.0,754.282200,0.000000,0.000000,8447.750707,14232.631583,58
56,362374.0,101,0.004746,495.489572,James Lick Freeway,298081.0,115146.0,-122.407687,37.740903,-122.405078,...,11101.801669,6911.867022,0.000000,0.0,2664.503005,0.000000,0.000000,8447.750707,14232.631583,58
43,362374.0,101,0.004746,495.489572,James Lick Freeway,298081.0,115146.0,-122.407687,37.740903,-122.405078,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,14595.416779,16765.092125,13
50,362374.0,101,0.004746,495.489572,James Lick Freeway,298081.0,115146.0,-122.407687,37.740903,-122.405078,...,3534.528300,563.375675,0.000000,0.0,0.000000,0.000000,0.000000,14595.416779,16765.092125,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,362441.0,101,0.010344,915.194582,John F Foran Freeway,106068.0,269793.0,-122.414657,37.732371,-122.424936,...,382.145750,1887.926008,151.196767,0.0,0.000000,107.366425,0.000000,26081.809560,27826.280992,63
121,362441.0,101,0.010344,915.194582,John F Foran Freeway,106068.0,269793.0,-122.414657,37.732371,-122.424936,...,2854.321564,7229.924492,1712.746385,0.0,0.000000,1424.295594,0.000000,26081.809560,27826.280992,63
110,362441.0,101,0.010344,915.194582,John F Foran Freeway,106068.0,269793.0,-122.414657,37.732371,-122.424936,...,308.570914,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,51114.152230,59753.003525,386
116,362441.0,101,0.010344,915.194582,John F Foran Freeway,106068.0,269793.0,-122.414657,37.732371,-122.424936,...,10438.545517,3514.493275,183.352067,0.0,1326.465367,152.755975,909.294600,51114.152230,59753.003525,386
